In [1]:
import nltk
import json
import logging
import sys
import plac
import logging
import argparse
import os
import json
import pickle
import pandas as pd

In [2]:
import spacy

nlp = spacy.blank("en")  # create blank Language class
print("Created blank 'en' model")

if 'ner' not in nlp.pipe_names :
    ner = nlp.create_pipe('ner')
    nlp.add_pipe(ner, last=True)
else :
    ner = nlp.get_pipe("ner")

Created blank 'en' model


In [3]:
lines=[]
input_file="C:/Users/jaysh/OneDrive/Desktop/training.json"
with open(input_file, 'r') as f:
    lines = f.readline()
print(lines)

{"content" :"mild patient: fever, respiratory and other symptoms, the manifestation of pneumonia can be seen on imaging.", "annotation": [{"label":["condition"],"points":[{"text":"fever","start":14,"end":18}]},{"label":["condition"],"points":[{"text":"respiratory","start":21,"end":31}]},{"label":["condition"],"points":[{"text":"symptoms","start":43,"end":50}]},{"label":["condition"],"points":[{"text":"pneumonia","start":74,"end":82}]},{"label":["procedure"],"points":[{"text":"imaging","start":99,"end":105}]}]}



In [4]:
def main(input_file="C:/Users/jaysh/OneDrive/Desktop/training.json", output_file="trainingNerMed_plac.pkl"):
    try:
        training_data = []
        lines=[]
        with open(input_file, 'r') as f:
            lines = f.readlines()

        for line in lines:
            data = json.loads(line)
            text = data['content']
            entities = []
            for annotation in data['annotation']:
                point = annotation['points'][0]
                labels = annotation['label']
                if not isinstance(labels, list):
                    labels = [labels]

                for label in labels:
                    entities.append((point['start'], point['end'] + 1 ,label))


            training_data.append((text, {"entities" : entities}))

        print(training_data)

        with open(output_file, 'wb') as fp:
            pickle.dump(training_data, fp)

    except Exception as e:
        logging.exception("Unable to process " + input_file + "\n" + "error = " + str(e))
        return None
if __name__ == '__main__':
    main()

[('mild patient: fever, respiratory and other symptoms, the manifestation of pneumonia can be seen on imaging.', {'entities': [(14, 19, 'condition'), (21, 32, 'condition'), (43, 51, 'condition'), (74, 83, 'condition'), (99, 106, 'procedure')]})]


In [5]:
train_data= pd.read_pickle("trainingNerMed_plac.pkl")

In [6]:
train_data

[('mild patient: fever, respiratory and other symptoms, the manifestation of pneumonia can be seen on imaging.',
  {'entities': [(14, 19, 'condition'),
    (21, 32, 'condition'),
    (43, 51, 'condition'),
    (74, 83, 'condition'),
    (99, 106, 'procedure')]})]

In [7]:
# Building Model using Training data
import spacy
import random
def train_spacy(data,iterations):
    train_data = data
    nlp = spacy.blank('en')  # create blank Language class
    # create the built-in pipeline components and add them to the pipeline
    # nlp.create_pipe works for built-ins that are registered with spaCy
    if 'ner' not in nlp.pipe_names:
        ner = nlp.create_pipe('ner')
        nlp.add_pipe(ner, last=True)
       

    # add labels
    for _, annotations in train_data:
         for ent in annotations.get('entities'):
            ner.add_label(ent[2])

    # get names of other pipes to disable them during training
    other_pipes = [pipe for pipe in nlp.pipe_names if pipe != 'ner']
    with nlp.disable_pipes(*other_pipes):  # only train NER
        optimizer = nlp.begin_training()
        for itn in range(iterations):
            #print("Starting iteration " + str(itn))
            #random.shuffle(train_data)
            losses = {}
            for text, annotations in train_data:
                nlp.update(
                    [text],  # batch of texts
                    [annotations],  # batch of annotations
                    drop=0.2,  # dropout - make it harder to memorise data
                    sgd=optimizer,  # callable to update weights
                    losses=losses)
            #print(losses)
    return nlp

prd= train_spacy(train_data, 500)



In [8]:
# Save our trained Model
# modelfile = input("Input your model name:")
# prd.to_disk(modelfile)

In [9]:
test_text = "mild patient: fever, respiratory and other symptoms, the manifestation of pneumonia can be seen on imaging."
P2="severe patients: meet the definition of severe pneumonia (comply with any of the followings): Shortness of breath, RR≥30 bpm ; In a resting state : SPO2≤93% ; PaO2/FiO2≤300mmHg."
doc = prd(P2)
for ent in doc.ents:
    print(ent.text, ent.start_char, ent.end_char, ent.label_)
    #print(ent.text,ent.label_)

meet 17 21 condition
severe 40 46 condition
with 65 69 condition
breath 107 113 condition
SPO2≤93 148 155 condition
PaO2 159 163 condition
FiO2≤300mmHg 164 176 procedure


In [16]:
def main(input_file="C:/Users/jaysh/OneDrive/Desktop/training2.json", output_file="trainingNerMed2_plac.pkl"):
    try:
        training_data = []
        lines=[]
        with open(input_file, 'r') as f:
            lines = f.readlines()
        for line in lines:
            data = json.loads(line)
            
            
            text = data['content']
            entities = []
            for annotation in data['annotation']:
                point = annotation['points'][0]
                labels = annotation['label']
                if not isinstance(labels, list):
                    labels = [labels]

                for label in labels:
                    entities.append((point['start'], point['end'] + 1 ,label))


            training_data.append((text, {"entities" : entities}))

#         print(training_data)

        with open(output_file, 'wb') as fp:
            pickle.dump(training_data, fp)

    except Exception as e:
        logging.exception("Unable to process " + input_file + "\n" + "error = " + str(e))
        return None

if __name__ == '__main__':
    main()

In [17]:
train_data2= pd.read_pickle("trainingNerMed2_plac.pkl")
train_data2

[('mild patient: fever, respiratory and other symptoms, the manifestation of pneumonia can be seen on imaging.',
  {'entities': [(14, 19, 'condition'),
    (21, 32, 'condition'),
    (43, 51, 'condition'),
    (74, 83, 'condition'),
    (99, 106, 'procedure')]}),
 ('severe patients: meet the definition of severe pneumonia (comply with any of the followings): Shortness of breath, RR>=30 bpm; In a resting state : SPO2<=93%; PaO2/FiO2<=300mmHg.',
  {'entities': [(47, 56, 'condition'),
    (107, 113, 'condition'),
    (115, 125, 'value'),
    (148, 157, 'measurement'),
    (159, 177, 'measurement')]})]

In [18]:
prd2=train_spacy(train_data2,500)

In [23]:
Pkl_Filename = "Pickle_Ner_Model.pkl"  

with open(Pkl_Filename, 'wb') as file:  
    pickle.dump(prd2, file)

In [22]:
P2 = "severe patients: meet the definition of severe pneumonia (comply with any of the followings): Shortness of breath, RR≥30 bpm ; In a resting state : SPO2≤93%; PaO2/FiO2≤300mmHg."
doc = prd2(P2)
for ent in doc.ents:
    print(ent.text, ent.start_char, ent.end_char, ent.label_)
    #print(ent.text,ent.label_)

pneumonia 47 56 condition
breath 107 113 condition
RR≥30 bpm 115 124 value
SPO2≤93% 148 156 measurement
PaO2/FiO2≤300mmHg 158 175 measurement


In [15]:
doc

severe patients: meet the definition of severe pneumonia (comply with any of the followings): Shortness of breath, RR<= 30 bpm ; In a resting state : SPO2≤93%; PaO2/FiO2≤300mmHg.